<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/12_evaluate_text_similarity_with_language_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 言語モデルを用いたテキストの類似度評価


**ランタイムのタイプはGPUに設定しておく。**

## 言語モデルによるテキストの埋め込み
* 今回は、二種類の言語モデルを使ってテキストをembedし・・・
* その後、内積で類似度を求める。
  * 内積は、大きい方がより類似している、という尺度。

### 必要なライブラリのインストール

* `fugashi[unidic-lite]`はcl-tohoku/bert-base-japanese-v3を使うために必要。

In [ ]:
!pip install -q transformers fugashi[unidic-lite] sentence-transformers accelerate

## (A) cl-tohoku/bert-base-japanese-v3

* このモデルについては、下記を参照。
  * https://huggingface.co/cl-tohoku/bert-base-japanese-v3

* モデルをダウンロードする。

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("cl-tohoku/bert-base-japanese-v3")

* テキストをembedしてみる。

In [ ]:
corpus = [
    "これはりんごです。",
    "これはりんごですか？",
    "あれはりんごです。",
]

In [ ]:
embeddings = model.encode(corpus)

In [ ]:
import numpy as np

print(f"「{corpus[0]}」と「{corpus[1]}」の類似度={np.dot(embeddings[0], embeddings[1]):.3f}")
print(f"「{corpus[0]}」と「{corpus[2]}」の類似度={np.dot(embeddings[0], embeddings[2]):.3f}")

## (B) intfloat/multilingual-e5-small
* E5というモデルについては、下記を参照。
  * https://huggingface.co/intfloat/multilingual-e5-base
  * https://hironsan.hatenablog.com/entry/2023/07/05/073150

* モデルをダウンロードする。

In [ ]:
from transformers import AutoTokenizer, AutoModel

model_name = "intfloat/multilingual-e5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).eval().to("cuda")

* テキストをembedするために、average poolingを自前で実装する。

In [ ]:
def average_pool(last_hidden_states, attention_mask):
  last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
  return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [ ]:
corpus = [
    "これはりんごです。",
    "これはりんごですか？",
    "あれはりんごです。",
]

In [ ]:
batch_dict = tokenizer(corpus, max_length=512, padding=True, truncation=True, return_tensors='pt').to("cuda")
with torch.no_grad():
  outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask']).cpu()

In [ ]:
import torch

print(f"「{corpus[0]}」と「{corpus[1]}」の類似度={torch.dot(embeddings[0], embeddings[1]):.3f}")
print(f"「{corpus[0]}」と「{corpus[2]}」の類似度={torch.dot(embeddings[0], embeddings[2]):.3f}")

## (C) elyza/ELYZA-japanese-Llama-2-7b-fast

* Google Colab無料版だと・・・
  * ローカルのキャッシュからモデルをロードするのに1分半ぐらいかかる。
* 手元にあるRTX4090搭載ゲーミングPCだと・・・
  * ローカルのキャッシュからモデルをロードするのに10秒未満。

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

model_name = "elyza/ELYZA-japanese-Llama-2-7b-fast"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
).eval()

In [ ]:
model.hf_device_map

In [ ]:
corpus = [
    "これはりんごです。",
    "これはりんごですか？",
    "あれはりんごです。",
]

In [ ]:
batch_dict = tokenizer(corpus, max_length=512, padding=True, truncation=True, return_tensors='pt').to("cuda")
with torch.no_grad():
  outputs = model(**batch_dict)

In [ ]:
def average_pool(last_hidden_states, attention_mask):
  last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
  return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [ ]:
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask']).cpu()

In [ ]:
import torch

embeddings = embeddings.type(torch.float32)
print(f"「{corpus[0]}」と「{corpus[1]}」の類似度={torch.dot(embeddings[0], embeddings[1]).item():.3f}")
print(f"「{corpus[0]}」と「{corpus[2]}」の類似度={torch.dot(embeddings[0], embeddings[2]).item():.3f}")

In [ ]:
embeddings.shape

In [ ]:
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz

In [ ]:
import re
import tarfile

tar_fname = "ldcc-20140209.tar.gz"

def read_title(f):
  next(f) # URL
  next(f) # タイムスタンプ
  title = next(f) # 3行目を返す：タイトル
  title = title.decode('utf-8')
  brackets_tail = re.compile('【[^】]*】$')
  brackets_head = re.compile('^【[^】]*】')
  return re.sub(brackets_head, "", re.sub(brackets_tail, "", title))[:-1]

corpus = []
with tarfile.open(tar_fname) as tf:
  for item in tf:
    if "LICENSE.txt" in item.name:
      continue
    if len(item.name.split('/')) < 3:
      continue
    if not item.name.endswith(".txt"):
      continue
    fname = item.name
    # 今回はクラス名は要らない
    #class_name = fname.split('/')[1]
    f = tf.extractfile(fname)
    title = read_title(f)
    corpus.append(title)

In [ ]:
len(corpus)

In [ ]:
def average_pool(last_hidden_states, attention_mask):
  last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
  return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

* RTX4090ゲーミングPCだと`batch_size=64`で動く。
  * 全データのembeddingは35秒で終わる。

In [ ]:
offset = 0
batch_size = 8
embeddings_list = list()
while offset < len(corpus):
  batch_dict = tokenizer(
      corpus[offset:offset+batch_size],
      max_length=128,
      padding=True, truncation=True, return_tensors='pt'
      ).to("cuda")
  with torch.no_grad():
    last_hidden_state = model(**batch_dict).last_hidden_state.cpu()
  embeddings = average_pool(last_hidden_state, batch_dict['attention_mask'].cpu())
  embeddings_list.append(embeddings)
  offset += batch_size

In [ ]:
embeddings = torch.concat(embeddings_list)
embeddings.shape

In [ ]:
torch.save(embeddings.type(torch.float32), "livedoor_ELYZA.pt")

In [ ]:
torch.dot(embeddings[0], embeddings[1]).item()